# Wahldaten-Crawler für alle Bundestags-, Landtags- und Europawahlen
Dieses Skript lädt Wahlergebnisse für alle Wahlen in Deutschland aus dem [Wahlarchiv der Tagesschau](https://wahl.tagesschau.de/wahlen/chronologie/chronologie.shtml) herunter. Diese werden in einem JSON-Format im aktuellen Verzeichnis gespeichert.

Das Ausgabeformat ist eine Liste mit Objekten. Jedes Objekt hat folgende Felder:

| Feld       | Beschreibung                                              |
|------------|-----------------------------------------------------------|
| title      | Titel der Wahl, z.B. Bundestagswahl 1990                  |
| url        | URL der Quelle                                            |
| date       | Datum der Wahl in ISO-8601                                |
| territory  | Bundesland, "Deutschland", "BRD", "DDR", oder "Europa"    |
| kind       | Art der Wahl, z.B. Landtagswahl                           |
| government | Liste der Parteien, die die Regierung stellen, als Kürzel |
| turnout    | Wahlbeteiligung in Prozent (optional)                     |
| results    | Wahlergebnisse der einzelnen Parteien                     |

Das Feld `results` enthält ein Objekt mit einem Schlüssel für jedes Parteikürzel. Der Wert ist jeweils ein Objekt mit folgenden Feldern:

| Feld      | Beschreibung                                        |
|-----------|-----------------------------------------------------|
| pct       | Wahlergebnis mit ein oder zwei Nachkommastellen     |
| votes     | Anzahl Stimmen (optional)                           |
| long_name | Voller Name oder Beschreibung der Partei (optional) |
| color     | Hexcode der Parteifarbe (optional)                  |

## Quellcode

Zunächst wird aus einer Übersichtsseite eine Liste der URLs aller Wahl-Seiten generiert.

In [1]:
import requests
from bs4 import BeautifulSoup

URL_BASE = "https://wahl.tagesschau.de/wahlen/"

overview_raw = requests.get(URL_BASE + "chronologie/chronologie.shtml")
page = BeautifulSoup(overview_raw.content, 'html.parser')

election_urls = []
for listing in page.find_all('ul', {"class": "list"}):
    for entry in listing.find_all('a'):
        election_urls.append({
            "title": entry.text,
            "url": URL_BASE + entry["href"][3:]
        })
        
print(f"Found {len(election_urls)} elections")

Found 270 elections


Als nächstes werden die Wahlseiten jeder Wahl geladen und die Wahlergebnisse und Metadaten extrahiert. Mehrere Hilfs-Funktionen extrahieren Metadaten aus der Seite.

In [35]:
import json
from datetime import datetime
from time import sleep

DATE_FORMAT = "%d.%m.%Y"
TIMESTAMP = datetime.strftime(datetime.now(), "%y%m%d")
OUT_FNAME = f"elections_germany.{TIMESTAMP}.json"

def get_metadata(base_title):
    try:
        date_text, kind, territory = election["title"].split(" ")
        year = date_text.split(".")[-1]
        title = f"{kind} {territory} {year}"
    except ValueError:
        date_text, kind = election["title"].split(" ")
        year = int(date_text.split(".")[-1])
        if kind == "Europawahl":
            territory = "Europawahl"
            title = f"{kind} in Deutschland {year}"
        elif kind == "Bundestagswahl":
            if year >= 1990:
                territory = "Deutschland"
            else:
                territory = "BRD"
            title = f"{kind} {year}"
        elif kind == "Volkskammerwahl":
            territory = "DDR"
            title = f"{kind} {year}"
            
    date = datetime.strptime(date_text, DATE_FORMAT).isoformat()

    return title, date, kind, territory

def select_government(page):
    government = None
    gov_raw = page.find("div", {"class": "regierung"})
    if gov_raw:
        government = [el.get_text() for el in gov_raw.find_all("span")[1:]]
    return government

def select_turnout(page):
    turnout = None
    mod_stoppers = page.find_all("div", {"class": "modStopper"})
    for ms in mod_stoppers:
        if "Wahlbeteiligung" in ms.get_text():
            turnout = float(ms.get_text().strip().split(" ")[1][:-1].replace(",", "."))
    return turnout

def select_party_info(glossary, party):
    if glossary is None:
        return {}
    
    candidates = [a for a in gloss.find_all("div", {"class": "glossarText"}) if party in a.get_text() ]

    rv = {}
    if len(candidates) == 1:
        long_name = candidates[0].get_text().split(":")[1].strip()
        color = candidates[0].previousSibling.previousSibling["style"].split("#")[1][:6]
        rv["long_name"] = long_name
        # #707173 is a placeholder grey color         
        if color != "707173":
            rv["color"] = "#" + color

    return rv
    
def extract_results_from_tables(tables, gloss):
    results = {}
    for t in tables:
        for row in t.find_all("tr", {"class": "row"}):
            party = row.find("td", {"class": "labelshort"}).get_text()
            results[party] = {
                "pct": float(row.find("td", {"class": "perc"}).get_text().replace(",", ".")),
                "votes": int(row.find("td", {"class": "votes"}).get_text().replace(".", ""))
            }
            results[party].update(select_party_info(gloss, party))
    return results
        
# Extract results from image
def extract_results_alt(page, gloss):
    # only small party results are shown in a table
    # find big party results  
    def extract_from_image(page, searchstring):
        els = page.find_all(lambda tag: tag.has_attr("alt") and searchstring in tag["alt"])
        contents = els[0]["alt"]
        contents_1, source = contents.split("Quelle:")
        _, contents_2 = contents_1.split("%:")
        return contents_2.split(";")
        
    try:
        results = extract_from_image(page, "Ergebnis")
    except ValueError:
        results = extract_from_image(page, "Endergebnis")
    except ValueError:
        print(page.find_all(lambda tag: tag.has_attr("alt") and "Ergebnis" in tag["alt"]))
        raise ValueError("No election results could be found on page!")
    rv = {}
    for result in results:
        try:
            party, pct = result.strip().split(" ")
            rv[party] = {
                "pct": float(pct.replace(",", "."))
            }
            rv[party].update(select_party_info(gloss, party))

        except ValueError:
            pass
    if "Andere" in rv.keys():
        del rv["Andere"]
    if "Sonstige" in rv.keys():
        del rv["Sonstige"]
    return rv

def save_results(elections):
    with open(OUT_FNAME, "w") as f:
        json.dump(sorted(elections, key=lambda x: x['title']), f, indent=2, ensure_ascii=False)

elections = []
for i, election in enumerate(election_urls):
    title, date, kind, territory = get_metadata(election["title"])        
    print(str(i) + " - " + title)
    d = requests.get(election["url"])
    page = BeautifulSoup(d.content, 'html.parser')
    
    government = select_government(page)
    turnout = select_turnout(page)

    entry = {}
    entry["title"] = title
    entry["url"] = election["url"]
    entry["date"] = date
    entry["territory"] = territory
    entry["kind"] = kind
    entry["government"] = government
    entry["turnout"] = turnout
    
    try:
        gloss_heading = page.find("h3", {"class": "glossar"})
        gloss = gloss_heading.next_sibling.next_sibling.find("ul")
    except AttributeError:
        print("\tNo glossary here")
        gloss = None
    
    results = {}
    try:
        tableA, tableB = page.find_all("table", {"class": "fivepercentX"})
        results = extract_results_from_tables([tableA, tableB], gloss)
    except ValueError:
        print("\tNo total vote count for parties >5%")
        results = extract_results_alt(page, gloss)
        tableB = page.find("table", {"class": "fivepercent"})
        resultsB = extract_results_from_tables([tableB], gloss) if tableB else {}
        results.update(resultsB)
    entry["results"] = results
    elections.append(entry)
    
    # sanity check (with rounding errors)    
    assert sum([p["pct"] for p in results.values()]) <= 101.0
    
    save_results(elections)
    
    # out of respect
    sleep(.5)

0 - Landtagswahl Thüringen 2019
1 - Landtagswahl Brandenburg 2019
2 - Landtagswahl Sachsen 2019
3 - Europawahl in Deutschland 2019
4 - Bürgerschaftswahl Bremen 2019
5 - Landtagswahl Hessen 2018
6 - Landtagswahl Bayern 2018
7 - Landtagswahl Niedersachsen 2017
8 - Bundestagswahl 2017
9 - Landtagswahl Nordrhein-Westfalen 2017
10 - Landtagswahl Schleswig-Holstein 2017
11 - Landtagswahl Saarland 2017
12 - Abgeordnetenhauswahl Berlin 2016
13 - Landtagswahl Mecklenburg-Vorpommern 2016
14 - Landtagswahl Sachsen-Anhalt 2016
15 - Landtagswahl Rheinland-Pfalz 2016
16 - Landtagswahl Baden-Württemberg 2016
17 - Bürgerschaftswahl Bremen 2015
18 - Bürgerschaftswahl Hamburg 2015
19 - Landtagswahl Thüringen 2014
	No total vote count for parties >5%
20 - Landtagswahl Brandenburg 2014
	No total vote count for parties >5%
21 - Landtagswahl Sachsen 2014
	No total vote count for parties >5%
22 - Europawahl in Deutschland 2014
	No total vote count for parties >5%
23 - Landtagswahl Hessen 2013
	No total vote 

123 - Landtagswahl Brandenburg 1990
	No total vote count for parties >5%
124 - Landtagswahl Nordrhein-Westfalen 1990
	No total vote count for parties >5%
125 - Landtagswahl Niedersachsen 1990
	No total vote count for parties >5%
126 - Volkskammerwahl 1990
	No total vote count for parties >5%
127 - Landtagswahl Saarland 1990
	No total vote count for parties >5%
128 - Europawahl in Deutschland 1989
	No total vote count for parties >5%
129 - Abgeordnetenhauswahl Berlin 1989
	No total vote count for parties >5%
130 - Landtagswahl Schleswig-Holstein 1988
	No total vote count for parties >5%
131 - Landtagswahl Baden-Württemberg 1988
	No total vote count for parties >5%
132 - Landtagswahl Schleswig-Holstein 1987
	No total vote count for parties >5%
133 - Bürgerschaftswahl Bremen 1987
	No total vote count for parties >5%
134 - Landtagswahl Rheinland-Pfalz 1987
	No total vote count for parties >5%
135 - Bürgerschaftswahl Hamburg 1987
	No total vote count for parties >5%
136 - Landtagswahl Hesse

	No total vote count for parties >5%
235 - Abgeordnetenhauswahl Berlin 1954
	No total vote count for parties >5%
236 - Landtagswahl Hessen 1954
	No total vote count for parties >5%
237 - Landtagswahl Bayern 1954
	No total vote count for parties >5%
238 - Landtagswahl Schleswig-Holstein 1954
	No total vote count for parties >5%
239 - Landtagswahl Nordrhein-Westfalen 1954
	No total vote count for parties >5%
240 - Bürgerschaftswahl Hamburg 1953
	No total vote count for parties >5%
241 - Bundestagswahl 1953
	No total vote count for parties >5%
242 - Landtagswahl Saarland 1952
	No total vote count for parties >5%
243 - Landtagswahl Baden-Württemberg 1952
	No total vote count for parties >5%
244 - Bürgerschaftswahl Bremen 1951
	No total vote count for parties >5%
245 - Landtagswahl Niedersachsen 1951
	No total vote count for parties >5%
246 - Landtagswahl Rheinland-Pfalz 1951
	No total vote count for parties >5%
247 - Abgeordnetenhauswahl Berlin 1950
	No total vote count for parties >5%
248